In [4]:
import os
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

factors = ['Beta60', 'OperatingRevenueGrowRate', 'NetProfitGrowRate', 'NetCashFlowGrowRate', 'NetProfitGrowRate5Y', 'TVSTD20',
           'TVSTD6', 'TVMA20', 'TVMA6', 'BLEV', 'MLEV', 'CashToCurrentLiability', 'CurrentRatio', 'REC', 'DAREC', 'GREC',
           'DASREV', 'SFY12P', 'LCAP', 'ASSI', 'LFLO', 'TA2EV', 'PEG5Y', 'PE', 'PB', 'PS', 'SalesCostRatio', 'PCF', 'CETOP',
           'TotalProfitGrowRate', 'CTOP', 'MACD', 'DEA', 'DIFF', 'RSI', 'PSY', 'BIAS10', 'ROE', 'ROA', 'ROA5', 'ROE5',
           'DEGM', 'GrossIncomeRatio', 'ROECut', 'NIAPCut', 'CurrentAssetsTRate', 'FixedAssetsTRate', 'FCFF', 'FCFE', 'PLRC6',
           'REVS5', 'REVS10', 'REVS20', 'REVS60', 'HSIGMA', 'HsigmaCNE5', 'ChaikinOscillator', 'ChaikinVolatility', 'Aroon',
           'DDI', 'MTM', 'MTMMA', 'VOL10', 'VOL20', 'VOL5', 'VOL60', 'RealizedVolatility', 'DASTD', 'DDNSR', 'Hurst']

df_ori = pd.read_csv('dataset2_factorRank10.csv')
df_ori.tail()

,Unnamed: 0,Unnamed: 0.1,ticker,tradeDate,Beta60,OperatingRevenueGrowRate,NetProfitGrowRate,NetCashFlowGrowRate,NetProfitGrowRate5Y,TVSTD20,...,RealizedVolatility,DASTD,DDNSR,Hurst,next_month_end,abs_return,active_return,industryName1,label,year
85743,85743,85743,603228,20171031,9,7,8,1,5,10,...,2,6,3,5,20171130,-0.079490,-0.079290,电子,0,2017
85744,85744,85744,600996,20171031,10,5,9,2,4,6,...,6,4,1,6,20171130,-0.110035,-0.109835,传媒,-1,2017
85745,85745,85745,601200,20171031,6,4,3,5,1,1,...,1,1,1,8,20171130,-0.116245,-0.116045,公用事业,-1,2017
85746,85746,85746,603225,20171031,4,4,1,2,2,8,...,1,3,1,8,20171130,-0.077162,-0.076962,化工,0,2017
85747,85747,85747,601020,20171031,2,3,2,9,3,10,...,3,2,1,9,20171130,-0.052632,-0.052432,有色金属,0,2017


In [3]:
tdate = np.unique(df_ori['tradeDate'].values)
l = len(tdate)
l

130

In [ ]:
def rf_rolling(df_ori,trainlen,testlen):
     for i in range(0,epoch+1):
        mark1 = tdate[i]
        mark2 = tdate[batchlen-1+i]
        batch = df_ori[(df_ori['tradeDate']>=mark1) & (df_ori['tradeDate']<mark2)]
        data_batch.append(batch)

    def train_test_dev(i):
        batch = data_batch[i]
        tdate_sub = np.unique(batch['tradeDate'].values)

        cut = tdate_sub[trainlen]
        test_df = batch[batch['tradeDate'] >=cut]
        train_df = batch[batch['tradeDate'] <cut]

        print('cut:',cut)

        # 用于训练的数据都屏蔽掉label为0的情况,并把label为-1的替换为0
        train_df = train_df[train_df['label']!=0]
        train_df.loc[:,'label'] = train_df.loc[:,'label'].apply(lambda x:0 if x ==-1 else x)
        
        return train_df,test_df

    def train_predict(i):
        
        train_df,test_df = train_test_dev(i)
        train_x = train_df[factors]
        train_y = train_df['label']
        test_x = test_df[factors]

        rf = RandomForestClassifier(n_estimators = 50,min_samples_split = 50,min_samples_leaf =13,\
                                    n_jobs = 8,max_depth = 14)
        rf.fit(train_x,train_y)
        rf.predict(test_x)
        return predict_score
    
    
    batchlen = trainlen+testlen
    epoch = l-1-batchlen
    data_batch = []
    tdate = np.unique(df_ori['tradeDate'].values)
    
    df = df_ori[df_ori['tradeDate']>=tdate[batchlen]]
    i = 0
    new_df = None
    for tdate,group in df.groupby('tradeDate'):
        print('round:',i)
        result = train_predict(i)
        print('tdate:',tdate)

        group.loc[:,'factor'] = result
        i += 1

        if new_df is None:
            new_df = group
        else:
            new_df = pd.concat([new_df, group],ignore_index=True)
        newdf = new_df.loc[:, ['ticker', 'tradeDate', 'label', 'factor']]
    return new_df

In [ ]:
new_df = rf_rolling(df_ori,24,2)
new_df.to_csv('rf_rolling_24_2.csv')

new_df = rf_rolling(df_ori,36,2)
new_df.to_csv('rf_rolling_36_2.csv')

new_df = rf_rolling(df_ori,72,2)
new_df.to_csv('rf_rolling_72_2.csv')

new_df = rf_rolling(df_ori,12,2)
new_df.to_csv('rf_rolling_12_2.csv')